In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os.path import join
import time

%load_ext autoreload
%autoreload 2

In [2]:
OUT_PATH='/n/groups/price/martin/LDSPEC_data/UKBB_trait_ss50K'
DATA_PATH='/n/groups/price/martin/LDSPEC_data/UKBB_trait_from_Steven'

### Load data

In [3]:
# Get the ID list
df_fam = pd.read_csv('/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001/unrelated_337K_ss50K.txt',
                     sep=' ', index_col=False)
df_fam.columns = ['FID', 'IID']
id_list = list(df_fam['IID'])
print('n_sample=%d' % len(id_list))

n_sample=50377


In [4]:
# 64 independent traits
xl = pd.ExcelFile(DATA_PATH+'/Description_080419.xlsx')
df_64_inde_trait = xl.parse('64 Recommended indepent traits', )
inde_trait_list = list(df_64_inde_trait['Trait_Identifier'])

In [5]:
# UKBB phen file
df_phen=pd.read_csv(DATA_PATH+'/UKB_v3.061518.tab', sep='\t', index_col=False)
df_cov=pd.read_csv(DATA_PATH+'/ukb4777.processed_and_post.plinkPCs.tab.gz', 
                   sep='\t', index_col=False, compression='gzip')
df_phen_bc=pd.read_csv(DATA_PATH+'/UKB_biochemistry.051619.tab',
                       sep='\t', index_col=False)
df_cov_bc=pd.read_csv(DATA_PATH+'/UKB_biochemistry.051619.cov',
                       sep='\t', index_col=False)

### trait_list, .phen, and .cov files

In [8]:
trait_list = sorted([x for x in df_phen.columns if x not in ['FID', 'IID']])
print('# trait_list=%d'%len(trait_list))
temp_df = df_phen.loc[df_phen['FID'].isin(id_list)].copy()
# trait_list = [x for x in trait_list if (~temp_df[x].isna()).sum()>1e5] # filter for N>1e5
# print('# Remove N<1e5 traits, trait_list=%d'%len(trait_list))
trait_list_indpt = [x for x in trait_list if 'UKB_460K.%s'%x in inde_trait_list]
print('# Restrict to 64 indpt traits, trait_list_indpt=%d'%len(trait_list_indpt))

trait_list_bc = sorted([x for x in df_phen_bc.columns if x not in ['FID', 'IID']])
print('# trait_list_bc=%d'%len(trait_list_bc))
temp_df = df_phen_bc.loc[df_phen_bc['FID'].isin(id_list)].copy()
# trait_list_bc = [x for x in trait_list_bc if (~temp_df[x].isna()).sum()>1e5]
# print('# Remove N<1e5 traits, trait_list_bc=%d'%len(trait_list_bc))
trait_list_bc_indpt = [x for x in trait_list_bc if 'UKB_460K.%s'%x in inde_trait_list]
print('# Restrict to 64 indpt traits, trait_list_bc_indpt=%d'%len(trait_list_bc_indpt))

with open(OUT_PATH+'/trait_list.txt', 'w') as f:
    for x in trait_list:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_indpt.txt', 'w') as f:
    for x in trait_list_indpt:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_other.txt', 'w') as f:
    for x in [x for x in trait_list if x not in trait_list_indpt]:
        f.write('%s\n'%x)
        
with open(OUT_PATH+'/trait_list_bc.txt', 'w') as f:
    for x in trait_list_bc:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_bc_indpt.txt', 'w') as f:
    for x in trait_list_bc_indpt:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_bc_other.txt', 'w') as f:
    for x in [x for x in trait_list_bc if x not in trait_list_bc_indpt]:
        f.write('%s\n'%x)
        
with open(OUT_PATH+'/trait_list_all.txt', 'w') as f:
    for x in trait_list+trait_list_bc:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_all_indpt.txt', 'w') as f:
    for x in trait_list_indpt+trait_list_bc_indpt:
        f.write('%s\n'%x)
with open(OUT_PATH+'/trait_list_all_other.txt', 'w') as f:
    for x in [x for x in trait_list+trait_list_bc if x not in trait_list_indpt+trait_list_bc_indpt]:
        f.write('%s\n'%x)

# trait_list=93
# Restrict to 64 indpt traits, trait_list_indpt=23
# trait_list_bc=30
# Restrict to 64 indpt traits, trait_list_bc_indpt=9


In [9]:
# Write phen file
for trait in trait_list:
    temp_df = df_phen.loc[df_phen['FID'].isin(id_list), ['FID', 'IID', trait]].copy()
    temp_df.dropna(inplace=True)
    temp_df.to_csv(OUT_PATH + '/%s.phen'%trait, sep='\t', header=True, index=False)
    print('%s, n_sample=%d' % (trait, temp_df.shape[0]))
    
for trait in trait_list_bc:
    temp_df = df_phen_bc.loc[df_phen_bc['FID'].isin(id_list), ['FID', 'IID', trait]].copy()
    temp_df.dropna(inplace=True)
    temp_df.to_csv(OUT_PATH + '/%s.phen'%trait, sep='\t', header=True, index=False)
    print('%s, n_sample=%d' % (trait, temp_df.shape[0]))

blood_EOSINOPHIL_COUNT, n_sample=48268
blood_HIGH_LIGHT_SCATTER_RETICULOCYTE_COUNT, n_sample=47994
blood_LYMPHOCYTE_COUNT, n_sample=48715
blood_MEAN_CORPUSCULAR_HEMOGLOBIN, n_sample=48706
blood_MEAN_PLATELET_VOL, n_sample=48832
blood_MEAN_SPHERED_CELL_VOL, n_sample=48005
blood_MONOCYTE_COUNT, n_sample=48554
blood_PLATELET_COUNT, n_sample=48763
blood_PLATELET_DISTRIB_WIDTH, n_sample=48776
blood_RBC_DISTRIB_WIDTH, n_sample=48593
blood_RED_COUNT, n_sample=48835
blood_WHITE_COUNT, n_sample=48780
bmd_HEEL_TSCOREz, n_sample=48965
body_BALDING1, n_sample=23131
body_BALDING4, n_sample=23131
body_BMIz, n_sample=50211
body_CORNEAL_HYSTERESIS_LEFT, n_sample=10550
body_CORNEAL_HYSTERESIS_RIGHT, n_sample=10564
body_HEIGHTz, n_sample=50262
body_LEFT_HANDED, n_sample=50368
body_WHRadjBMIz, n_sample=50273
bp_DIASTOLICadjMEDz, n_sample=46427
bp_SYSTOLICadjMEDz, n_sample=46427
cancer_ALL, n_sample=50377
cancer_BREAST, n_sample=50377
cancer_MELANOMA, n_sample=50377
cancer_PROSTATE, n_sample=50377
cov_EDU

### .resid.phen file

In [10]:
cov_list = ['cov_ASSESS_CENTER', 'cov_GENO_ARRAY', 'cov_SEX']
qcov_list = ['cov_AGE', 'cov_AGE_SQ'] + ['PC%d'%x for x in np.arange(1,21)] 

# Categorical covaraites 
temp_df_cov = df_cov[['FID', 'IID'] + cov_list]
temp_df_cov.index = temp_df_cov['IID']
temp_df_cov = pd.get_dummies(temp_df_cov[cov_list], columns=cov_list, drop_first=True)

# df_reg
df_reg = df_cov[['FID', 'IID'] + qcov_list]
df_reg.index = df_reg['IID']
df_reg = df_reg.join(temp_df_cov)
PC_list = [x for x in df_reg.columns if 'PC' in x]
df_reg[PC_list] *= np.sqrt(df_reg.shape[0])
df_reg['const'] = 1
df_reg.drop(['FID', 'IID'], axis=1, inplace=True)
df_reg = df_reg.astype(dtype=np.float32)

# Compute residual
for trait in trait_list:
    df_phen = pd.read_csv(OUT_PATH+'/%s.phen'%trait, sep='\t', header=0, index_col=False)
    df_phen.index = df_phen['IID']    
    sample_list = sorted(set(df_phen.index) & set(df_reg.index))
    print('%s, n_sample=%d, n_sample_retain=%d' % (trait, df_phen.shape[0], len(sample_list)))

    temp_df_reg = df_reg.loc[sample_list].copy()
    drop_list = [x for x in temp_df_reg if (temp_df_reg[x].std()<1e-2) & (x!='const')]
    if len(drop_list)>0:
        temp_df_reg = temp_df_reg.drop(columns=drop_list)
        print('    col dropped: %s'%', '.join(drop_list))
        
    v_y = df_phen.loc[sample_list][trait].values.astype(np.float32)
    mat_X = temp_df_reg.values.astype(np.float32)
    n_sample = v_y.shape[0]
    mat_xtx = np.dot(mat_X.T, mat_X)/n_sample
    mat_xty = np.dot(mat_X.T, v_y)/n_sample
    coef_ = np.linalg.solve(mat_xtx, mat_xty).reshape([-1])
    v_y_res = v_y - mat_X.dot(coef_)
    print('    var(v_y)=%0.4g, var(v_y_res)=%0.4g, R_square=%0.4g\n'
          %(v_y.var(), v_y_res.var(), 1- v_y_res.var()/v_y.var()))
    
    temp_df = pd.DataFrame( data = {
        "FID" : sample_list,
        "IID" : sample_list,
        trait : v_y_res / v_y_res.std()
    }
    )
    temp_df.to_csv(OUT_PATH + '/%s.resid.phen'%trait, sep='\t', header=True, index=False)

blood_EOSINOPHIL_COUNT, n_sample=48268, n_sample_retain=48268
    var(v_y)=0.9988, var(v_y_res)=0.9906, R_square=0.008209

blood_HIGH_LIGHT_SCATTER_RETICULOCYTE_COUNT, n_sample=47994, n_sample_retain=47994
    var(v_y)=0.9891, var(v_y_res)=0.985, R_square=0.004181

blood_LYMPHOCYTE_COUNT, n_sample=48715, n_sample_retain=48715
    var(v_y)=0.9859, var(v_y_res)=0.9744, R_square=0.01166

blood_MEAN_CORPUSCULAR_HEMOGLOBIN, n_sample=48706, n_sample_retain=48706
    var(v_y)=0.9485, var(v_y_res)=0.939, R_square=0.00996

blood_MEAN_PLATELET_VOL, n_sample=48832, n_sample_retain=48832
    var(v_y)=0.9916, var(v_y_res)=0.9867, R_square=0.004991

blood_MEAN_SPHERED_CELL_VOL, n_sample=48005, n_sample_retain=48005
    var(v_y)=0.9735, var(v_y_res)=0.9685, R_square=0.005083

blood_MONOCYTE_COUNT, n_sample=48554, n_sample_retain=48554
    var(v_y)=0.9895, var(v_y_res)=0.984, R_square=0.005525

blood_PLATELET_COUNT, n_sample=48763, n_sample_retain=48763
    var(v_y)=1.002, var(v_y_res)=0.9988, R_squar

other_MORNINGPERSON, n_sample=45002, n_sample_retain=45002
    var(v_y)=0.2326, var(v_y_res)=0.2299, R_square=0.01186

other_NAP_TIME, n_sample=50361, n_sample_retain=50361
    var(v_y)=0.2458, var(v_y_res)=0.2366, R_square=0.03728

other_NO_WHEAT, n_sample=50251, n_sample_retain=50251
    var(v_y)=0.02601, var(v_y_res)=0.0259, R_square=0.004185

other_SODIUM_URINE, n_sample=48859, n_sample_retain=48859
    var(v_y)=1871, var(v_y_res)=1703, R_square=0.08979

other_TIME_TV, n_sample=50044, n_sample_retain=50044
    var(v_y)=2.814, var(v_y_res)=2.637, R_square=0.06269

other_WALKING, n_sample=50132, n_sample_retain=50132
    var(v_y)=0.2399, var(v_y_res)=0.2339, R_square=0.0251

parents_ALZHEIMER, n_sample=49098, n_sample_retain=49098
    var(v_y)=0.1026, var(v_y_res)=0.1005, R_square=0.0204

parents_BOWEL_CANCER, n_sample=49098, n_sample_retain=49098
    var(v_y)=0.06232, var(v_y_res)=0.06205, R_square=0.00439

parents_BREAST_CANCER, n_sample=49098, n_sample_retain=49098
    var(v_y)=0.

In [11]:
# trait_bc
cov_list = ['cov_ASSESS_CENTER', 'cov_GENO_ARRAY', 'cov_SEX']
qcov_list = ['cov_AGE', 'cov_AGE_SQ', 'biochemistry_dilutionfactor'] + ['PC%d'%x for x in np.arange(1,21)] 

# Categorical covaraites 
temp_df_cov = df_cov_bc[['FID', 'IID'] + cov_list]
temp_df_cov.index = temp_df_cov['IID']
temp_df_cov = pd.get_dummies(temp_df_cov[cov_list], columns=cov_list, drop_first=True)

# df_reg
df_reg = df_cov_bc[['FID', 'IID'] + qcov_list].copy()
df_reg.dropna(inplace=True)
df_reg.index = df_reg['IID']
df_reg = df_reg.join(temp_df_cov)
PC_list = [x for x in df_reg.columns if 'PC' in x]
df_reg[PC_list] *= np.sqrt(df_reg.shape[0])
df_reg['const'] = 1
df_reg.drop(['FID', 'IID'], axis=1, inplace=True)
df_reg = df_reg.astype(dtype=np.float32)

# Compute residual
for trait in trait_list_bc:
    df_phen = pd.read_csv(OUT_PATH+'/%s.phen'%trait, sep='\t', header=0, index_col=False)
    df_phen.index = df_phen['IID']    
    sample_list = sorted(set(df_phen.index) & set(df_reg.index))
    print('%s, n_sample=%d, n_sample_retain=%d' % (trait, df_phen.shape[0], len(sample_list)))

    temp_df_reg = df_reg.loc[sample_list].copy()
    drop_list = [x for x in temp_df_reg if (temp_df_reg[x].std()<1e-2) & (x!='const')]
    if len(drop_list)>0:
        temp_df_reg = temp_df_reg.drop(columns=drop_list)
        print('    col dropped: %s'%', '.join(drop_list))
        
    v_y = df_phen.loc[sample_list][trait].values.astype(np.float32)
    mat_X = temp_df_reg.values.astype(np.float32)
    n_sample = v_y.shape[0]
    mat_xtx = np.dot(mat_X.T, mat_X)/n_sample
    mat_xty = np.dot(mat_X.T, v_y)/n_sample
    coef_ = np.linalg.solve(mat_xtx, mat_xty).reshape([-1])
    v_y_res = v_y - mat_X.dot(coef_)
    print('    var(v_y)=%0.4g, var(v_y_res)=%0.4g, R_square=%0.4g\n'
          %(v_y.var(), v_y_res.var(), 1- v_y_res.var()/v_y.var()))
    
    temp_df = pd.DataFrame( data = {
        "FID" : sample_list,
        "IID" : sample_list,
        trait : v_y_res / v_y_res.std()
    }
    )
    temp_df.to_csv(OUT_PATH + '/%s.resid.phen'%trait, sep='\t', header=True, index=False)

biochemistry_AlanineAminotransferase, n_sample=47671, n_sample_retain=47377
    var(v_y)=122.3, var(v_y_res)=110.6, R_square=0.09544

biochemistry_Albumin, n_sample=44032, n_sample_retain=43736
    col dropped: biochemistry_dilutionfactor
    var(v_y)=6.698, var(v_y_res)=6.387, R_square=0.04642

biochemistry_AlkalinePhosphatase, n_sample=47895, n_sample_retain=47601
    var(v_y)=515.9, var(v_y_res)=493.3, R_square=0.04367

biochemistry_ApolipoproteinA, n_sample=43812, n_sample_retain=43517
    col dropped: biochemistry_dilutionfactor
    var(v_y)=0.07232, var(v_y_res)=0.06057, R_square=0.1624

biochemistry_ApolipoproteinB, n_sample=47846, n_sample_retain=47552
    var(v_y)=0.05679, var(v_y_res)=0.05595, R_square=0.01478

biochemistry_AspartateAminotransferase, n_sample=47587, n_sample_retain=47298
    var(v_y)=53.11, var(v_y_res)=49.73, R_square=0.0637

biochemistry_Calcium, n_sample=43969, n_sample_retain=43674
    col dropped: biochemistry_dilutionfactor
    var(v_y)=0.008303, var(v_

### Sanity check 

In [12]:
PHEN_FILE = '/n/groups/price/martin/LDSPEC_data/UKBB_trait_ss50K/@.resid.phen'
PHEN_FILE_OLD = '/n/groups/price/martin/LDSPEC_data/UKBB_trait/@.resid.phen'
# PHEN_FILE_OLD = '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001/UKBB_trait/@.resid.phen'
# PHEN_FILE = "/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/UKBB_trait/@.resid.phen"
# PHEN_FILE_OLD = "/n/groups/price/martin/data_GDREG/WES_50K_imp/UKBB_trait/@.resid.phen"
# for trait in trait_list + trait_list_bc : 
for trait in trait_list_indpt + trait_list_bc_indpt : 
    temp_df1 = pd.read_csv(PHEN_FILE.replace('@', trait), sep='\t', header=0, index_col=False)
    temp_df1.index = temp_df1['IID']
    temp_df2 = pd.read_csv(PHEN_FILE_OLD.replace('@', trait), sep='\t', header=0, index_col=False)
    temp_df2.index = temp_df2['IID']
#     temp_df2.index = [int(x.split('_')[1]) for x in temp_df2['IID']]
    sample_list = sorted(set(temp_df1.index) & set(temp_df2.index))
    print('%s n_sample1=%d, n_sample2=%d, n_overlap=%d'
          % (trait, temp_df1.shape[0], temp_df2.shape[0], len(sample_list)))
    
    temp_df1 = temp_df1.loc[sample_list]
    temp_df2 = temp_df2.loc[sample_list]
    
    print('    cor=%0.3f, mean_abs_err=%0.4g' % (
        np.corrcoef(temp_df1[trait], temp_df2[trait])[0,1], 
        np.absolute(temp_df1[trait]-temp_df2[trait]).mean()
    ))

blood_PLATELET_COUNT n_sample1=48763, n_sample2=326556, n_overlap=48763
    cor=1.000, mean_abs_err=0.02015
blood_RBC_DISTRIB_WIDTH n_sample1=48593, n_sample2=325330, n_overlap=48593
    cor=0.999, mean_abs_err=0.02748
blood_RED_COUNT n_sample1=48835, n_sample2=327144, n_overlap=48835
    cor=1.000, mean_abs_err=0.02264
blood_WHITE_COUNT n_sample1=48780, n_sample2=326659, n_overlap=48780
    cor=1.000, mean_abs_err=0.0207
bmd_HEEL_TSCOREz n_sample1=48965, n_sample2=327675, n_overlap=48965
    cor=1.000, mean_abs_err=0.0175
body_BALDING1 n_sample1=23131, n_sample2=155070, n_overlap=23131
    cor=0.999, mean_abs_err=0.03332
body_BMIz n_sample1=50211, n_sample2=336329, n_overlap=50211
    cor=1.000, mean_abs_err=0.02087
body_HEIGHTz n_sample1=50262, n_sample2=336694, n_overlap=50262
    cor=1.000, mean_abs_err=0.0217
body_WHRadjBMIz n_sample1=50273, n_sample2=336781, n_overlap=50273
    cor=1.000, mean_abs_err=0.02225
bp_DIASTOLICadjMEDz n_sample1=46427, n_sample2=310771, n_overlap=46427
